In [5]:
!python --version

Python 3.11.9


In [39]:
# Importimised
import pickle
import re
import os
import pandas as pd
import numpy as np
import sklearn as sk
import tensorflow as tf

In [40]:
print(f"pandas: {pd.__version__}") # pandas: 2.1.4
print(f"numpy: {np.__version__}") # numpy: 1.26.4
print(f"sklearn: {sk.__version__}") # sklearn: 1.2.2
print(f"tensorflow: {tf.__version__}") # tensorflow: 2.16.1

pandas: 2.1.4
numpy: 1.26.4
sklearn: 1.2.2
tensorflow: 2.16.1


### Ennustamine

Selles peatükis on võimalik kasutada mudelit muuttüüpide ennustamiseks sõnadele.

Järgmise koodiga võetakse häälestatud muuttüübi `MultiLabelBinarizer`. Eeldatakse, et fail on kättesaadav kaustas `Binarizers` failinimena `mlb_w.pkl`.

In [13]:
def get_mlb_w(df_algvormidega):
  filename = 'Binarizers/mlb_w.pkl'
  if os.path.exists(filename):
    with open(filename, 'rb') as file:
      mlb_w = pickle.load(file)
  else:
    print("Ei leidnud olemasolevat sõnaliigi vektoriseerijat MultiLabelBinarizer faili nimega mlb_w.pkl")
    return None
  return mlb_w

Järgmise koodiga võetakse häälestatud üldisema sõnaliigi `MultiLabelBinarizer`. Eeldatakse, et fail on kättesaadav kaustas `Binarizers` failinimena `mlb_s.pkl`.

In [14]:
def get_mlb_s_üldisem(df_algvormidega):
  pos_to_group = {
        "['A']": 'n',  # omadussõna - algvõrre (adjektiiv - positiiv), nii käänduvad kui käändumatud, nt kallis või eht, Adjective
        "['C']": 'n',  # omadussõna - keskvõrre (adjektiiv - komparatiiv), nt laiem, Comparative adjective
        "['D']": 'u',  # määrsõna (adverb), nt kõrvuti, Adverb
        "['G']": 'u',  # genitiivatribuut (käändumatu omadussõna), nt balti, Genitive attribute
        "['H']": 'n',  # pärisnimi, nt Edgar, Proper noun
        "['I']": 'u',  # hüüdsõna (interjektsioon), nt tere, Interjection
        "['J']": 'u',  # sidesõna (konjunktsioon), nt ja, Conjunction
        "['K']": 'u',  # kaassõna (pre/postpositsioon), nt kaudu, Pre/postposition
        "['N']": 'n',  # põhiarvsõna (kardinaalnumeraal), nt kaks, Cardinal numeral
        "['O']": 'n',  # järgarvsõna (ordinaalnumeraal), nt teine, Ordinal numeral
        "['P']": 'n',  # asesõna (pronoomen), nt see, Pronoun
        "['S']": 'n',  # nimisõna (substantiiv), nt asi, Noun
        "['U']": 'n',  # omadussõna - ülivõrre (adjektiiv - superlatiiv), nt pikim, Superlative adjective
        "['V']": 'v',  # tegusõna (verb), nt lugema, Verb
        "['X']": 'u',  # verbi juurde kuuluv sõna, millel eraldi sõnaliigi tähistus puudub, nt plehku, Adverb-like word used solely with a certain verb
        "['Y']": 'n',  # lühend, nt USA, Abbreviation or acronym
        "['Z']": 'u',  # lausemärk, nt -, /, ..., Punctuation
        # Siit alates lisaks statistikast leitud read
        "['S', 'S']": 'n',
        "['A', 'A']": 'n',
        "['S', 'I']": 'n',
        "['V', 'V']": 'v',
        "['D', 'K']": 'u',
        "['H', 'H']": 'n',
        "['P', 'P']": 'n',
        "['S', 'A']": 'n',
        "['K', 'D']": 'u',
        "['A', 'S']": 'n',
    }

  filename = 'Binarizers/mlb_s.pkl'

  if os.path.exists(filename):
    with open(filename, 'rb') as file:
      mlb_s = pickle.load(file)
  else:
    print("Ei leidnud olemasolevat sõnaliigi vektoriseerijat MultiLabelBinarizer faili nimega mlb_s.pkl")
    return None
  return mlb_s

Järgmise koodiga võetakse häälestatud sõnade TextVectorization. Eeldatakse, et failid on kättesaadavad kaustas `TextVectorizations` nimedena `text_vectorization_len_20_model_architecture.pkl` ja `text_vectorization_len_20_vocab.pkl`

In [16]:
def tekst_vect_failid():
  # Kaust, kuhu on kogutud kõik TextVectorization'id
  kaust = "./TextVectorizations/"

  vocab_file = kaust + "text_vectorization_vocab.pkl" # TextVectorization sõnastik

  # Kui on olemas failid, siis loe sisse
  if os.path.exists(vocab_file):
    with open(vocab_file, "rb") as f:
        vocab = pickle.load(f)

    tekst_vect = tf.keras.layers.TextVectorization(split="character",
                                                  output_mode="int",
                                                  output_sequence_length=20)
    tekst_vect.set_vocabulary(vocab)
  else:
    print("Ei leidnud olemasolevat TextVectorization sõnastikku text_vectorization_vocab.pkl")
    return None

  return tekst_vect

Järgmine kood on funktsioon muuttüüpide ennustamiseks.

Sõnamudeli kasutamiseks eeldatakse sisendiks järjendit sõnadest  kujul `[sõna_1, sõna_2, ...]`.

Sõnamudel sõnaliigiga kasutamiseks eeldatakse sisendiks järjendit sõnadest  kujul `[sõna_1, sõna_2, ...]` ja järjendit sõnaliikidest kujul `[sõnaliik_1, sõnaliik_2, ...]`.

In [41]:
def leia_muuttüüp(sõna, sõnaliik = ''):

  df_algvormidega = pd.read_csv("andmed_algvormidega.csv", header=0, keep_default_na=False)

  # Mudeli leidmine
  try:
    if sõnaliik: # Sõnaliigiga sõnamudel
      mudeli_path = './Sonaliigiga_sonamudel/sonaliigiga_sonamudel.keras'
    else: # Sõnamudel
      mudeli_path = './Sonamudel/sonamudel.keras'
    mudel = tf.keras.models.load_model(mudeli_path)
  except OSError as e:
    print(f"{mudeli_path} ei ole mudel")
    return

  # TextVectorization
  tekst_vect = tekst_vect_failid()
  if not tekst_vect:
    print("Paiguta text_vectorization_vocab.pkl TextVectorizations kausta")
    return
  X_w = tekst_vect(sõna)

  # Muuttüübid
  mlb_w = get_mlb_w(df_algvormidega)
  if not mlb_w:
    print("Paiguta mlb_w.pkl Binarizers kausta")

  # Sõnaliigid
  if sõnaliik:
    mlb_s = get_mlb_s_üldisem(df_algvormidega)
    if not mlb_s:
      print("Paiguta mlb_s.pkl Binarizers kausta")
      return
    sõnaliik = [[s] for s in sõnaliik]
    X_s = tf.convert_to_tensor(mlb_s.transform(sõnaliik)) # Ei tohi olla Tensor (sõna) ja mitte Tensor (sõnaliik) koos, sestap convert_to_tensor

  # Ennustamine
  if sõnaliik:
    sisend_tulemused = mudel([X_w, X_s], training=False)
  else:
    sisend_tulemused = mudel(X_w, training=False)

  # Tulemuste salvestamine
  tulemused = list()
  for tulemus in sisend_tulemused:
    indeks = np.argmax(tulemus)
    ennustatud_muuttüüp = mlb_w.classes_[indeks]
    ennustatud_muuttüüp = re.sub(r"[\[\]']", "", ennustatud_muuttüüp)#.split(',')
    # Teisendab muuttüübi andmetüübi täisarvuks
    #if len(ennustatud_muuttüüp) > 1:
    #  ennustatud_muuttüüp = [int(ennustatud_muuttüüp[0]), int(ennustatud_muuttüüp[1])]
    #else:
    #  ennustatud_muuttüüp = [int(ennustatud_muuttüüp[0])]
    tulemused.append(ennustatud_muuttüüp)

  return tulemused

Järgnev kood kasutab sõnamudelit muuttüübi ennustamiseks.

In [42]:
leia_muuttüüp(["vilistlasel", "sinist", "musta", "ja", "valget", "ülikoolis", "arukalt"])

e:\Anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


['12', '10', '22', '-', '6', '22', '2']

Järgnev kood kasutab sõnamudelit sõnaliigiga muuttüübi ennustamiseks.

In [36]:
leia_muuttüüp(["vilistlasel", "sinist", "musta", "ja", "valget", "ülikoolis", "arukalt"], ['n', 'n', 'n', 'u', 'n', 'n', 'u'])

e:\Anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 22 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


['12', '10', '22', '-', '1', '22', '-']

In [21]:
leia_muuttüüp(["sulgema", "sulen", "sulgen", "sule", "sulge", "sulgeda"])

e:\Anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


['28, 27', '28', '27', '22', '22', '28, 27']